In [1]:
from qm1.grid import *
from qm1.qmsystem import *

In [2]:
# set up a grid (with vanishing boundary conditions)
grid = UniformGrid(boundary_condition="vanishing", xmin=-20., xmax=20., num=250)
# define potential to use
potential = BarrierPot(xstart=-1., xstop=+2., vstep=-1.)
# set the quantum mechanical system
qsys = QMSystem(potential, grid)

In [3]:
from qm1.operators import *
# get predefined operators that act on the wave function
op_identity = IdentityOp(qsys.grid)
op_position = PositionOp(qsys.grid)
op_momentum = MomentumOp(qsys.grid)
op_hamilton = HamiltonOp(qsys)
op_potential= PotentialOp(qsys)
# make the operators more efficient
make_efficient([op_identity, op_position, op_momentum, op_potential, op_hamilton])

In [4]:
from qm1.eigensystem import Eigensystem
import pandas

# compute 10 eigenstates of the Hamiltonian
num_states = 10
eigsys = Eigensystem(qsys=qmsys, operator=op_hamilton)
eigsys.show('eigensystem.png')
# survey their observables
norms, positionExps, positionVars, momenta, energyExps, energyVars = [], [], [], [], [], []
for _i in range(num_states):
  norms.append(np.real(eigsys.eigstates[_i].expectation_value(op_identity)))
  positionExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_position)))
  positionVars.append(np.real(eigsys.eigstates[_i].variance(op_position)))
  momenta.append(np.real(eigsys.eigstates[_i].expectation_value(op_momentum)))
  energyExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_hamilton)))
  energyVars.append(np.real(eigsys.eigstates[_i].variance(op_hamilton)))

pandas.DataFrame([norms, positionExps, positionVars, momenta, energyExps, energyVars], ['norms','positions', 'positionVars', 'momenta', 'energies', 'energyVars'], range(num_states))

TypeError: __init__() missing 1 required positional argument: 'qsys'

In [ ]:
import matplotlib.pyplot as plt# plot ...
fig, ax1 = plt.subplots(figsize=(15,5))
fig.subplots_adjust(right=0.6)
colors = [plt.cm.tab10(i) for i in range(num_states)]
alphas = [ ((num_states-i)/num_states)**2. for i in range(num_states)]

# ... wavefunctions on ax1
ax1.set_title('eigenstates of the hamiltonian')
ax1.set_xlabel('position')
ax1.set_ylabel('wave function')
ax1.set_xlim((qsys.grid.xmin, qsys.grid.xmax))
for _i, (_eigs, _col, _a) in enumerate(zip(eigsys.eigstates, colors, alphas)):
  ax1.plot(qsys.grid.points, _eigs.func, label='state '+str(_i), alpha=_a, color=_col)
ax1.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))

# ... energy levels and potential on ax2
ax2 = ax1.twinx()
ax2.set_ylabel('energy scale / potential')
ax2.plot(qsys.grid.points, qsys.pot, "k")
for _energy, _col, _a in zip(energyExps, colors, alphas):
  ax2.axhline(_energy, color=_col, alpha=_a, ls='--')

plt.plot()